<a href="https://colab.research.google.com/github/ClearSpear/cameramouse/blob/master/handdetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mount Google Drive to save files

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd '/content/gdrive/My Drive/Stanford/EE292D/project/colab'

## Check remaining time on colab session:

In [ ]:
import time, psutil
Start = time.time()- psutil.boot_time()
Left= 12*3600 - Start
print('Time remaining for this session is: ', Left/3600)

## Download TF object detection API

In [ ]:
%tensorflow_version 1.x

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print(tf.__version__)

In [ ]:
tf.enable_eager_execution()

In [ ]:
!git clone https://github.com/tensorflow/models.git

In [ ]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
!pip install tf_slim

In [ ]:
%cd /content/gdrive/My Drive/Stanford/EE292D/project/colab/models/research/
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
import os
os.environ['PYTHONPATH'] += ':/content/gdrive/My Drive/Stanford/EE292D/project/colab/models/research/:/content/gdrive/My Drive/Stanford/EE292D/project/colab/models/research/slim'

In [ ]:
!python setup.py build
!python setup.py install

Test installation was successful:

In [ ]:
%cd /content/gdrive/My Drive/Stanford/EE292D/project/colab/models/research/object_detection/builders/
!python model_builder_test.py

## Setup to train hand detection model

In [ ]:
%cd /content/gdrive/My Drive/Stanford/EE292D/project/colab

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_11_06_2017.tar.gz
!tar -xvf ssd_mobilenet_v1_coco_11_06_2017.tar.gz

In [ ]:
%load_ext tensorboard
%tensorboard --logdir training/

## Train

In [ ]:
!python models/research/object_detection/train.py --logtostderr --train_dir=training/ --pipeline_config_path=training/ssd_mobilenet_v1_coco.config

## Convert saved model to frozen graph compatible with tflite

In [ ]:
H = 480
W = 640

In [ ]:
!python models/research/object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path=training/ssd_mobilenet_v1_coco.config \
--trained_checkpoint_prefix=training/model.ckpt-18937 \
--output_directory=tflite \
--add_postprocessing_op=true

## Convert the tflite model, quantizing using the representative dataset.

In [ ]:
gf = tf.GraphDef()   
m_file = open('tflite/tflite_graph.pb','rb')
gf.ParseFromString(m_file.read())

for n in gf.node:
    print( n.name )
    tensor = n.op # save the input and output tensor

In [ ]:
# A generator that provides a representative dataset
def representative_data_gen():
  dataset_list = tf.data.Dataset.list_files('egohands/train/*')
  for i in range(100):
    image = next(iter(dataset_list))
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [H, W])
    image = tf.cast(image / 255., tf.float32)
    image = tf.expand_dims(image, 0)
    yield [image]

input_tensors = ['normalized_input_image_tensor']
output_tensors = ['TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3']

converter = tf.lite.TFLiteConverter.from_frozen_graph('tflite/tflite_graph.pb', input_tensors, output_tensors, input_shapes={'normalized_input_image_tensor':[1,H,W,3]})
converter.allow_custom_ops = True
converter.optimizations = [tf.lite.Optimize.DEFAULT]
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
#converter.inference_output_type = tf.uint8
converter.representative_dataset = representative_data_gen
tflite_model = converter.convert()

with open("converted_model.tflite", "wb") as f:
  f.write(tflite_model)